# [Construct Binary Tree from Preorder and Inorder Traversal](https://leetcode.com/problems/construct-binary-tree-from-preorder-and-inorder-traversal/)
## Description
Build a binary tree from preorder and inorder traversal arrays.
## Strategy
Use recursion and map of inorder indexes. Reconstruct root from preorder and partition in inorder.

Both __preorder__ and __inorder__ give you ambiguous view of the binary tree.

**To rebuild the actual tree structure, you typically need:**

- **Preorder + Inorder** (most common approach)
- **Preorder + constraints** (like BST property)
- **Preorder + null markers** (like in serialization: `[1,2,null,null,3,null,null]`)

### Preorder

**Preorder reveals:**
- The **order** you'd visit nodes starting from root
- Which node is the **root** (always first element)
- Which nodes are **ancestors** of others

#### What Preorder Doesn't Tell You

**Preorder alone doesn't reveal:**
- **Left vs right** child relationships
- The actual **tree structure/shape**
- **Parent-child connections** beyond identifying the root

#### Example: Ambiguous Structure

**Given preorder: `[1, 2, 3]`**

This could represent any of these different trees:

```
Tree A:          Tree B:          Tree C:
   1                1                1
  /                  \              / \
 2                    2            2   3
/                      \
3                       3
```

All three trees produce the same preorder traversal!

### Inorder

**Inorder reveals:**
- The **sorted order** of nodes (for BSTs)
- **Left-to-right** ordering of nodes at each level
- Which nodes are in the **left vs right subtrees** of any given node

#### What Inorder Doesn't Tell You

**Inorder alone doesn't reveal:**
- Which node is the **root**
- The actual **tree structure/shape**
- **Parent-child relationships**
- **Depth/level** of nodes

#### Example: BST Property

**For a Binary Search Tree:**
```
Tree:        Inorder: [1, 2, 3, 4, 5, 6, 7]
    4
   / \
  2   6
 / \ / \
1  3 5  7
```

The inorder traversal gives you the **sorted sequence** - this is guaranteed for any valid BST!

#### Example: Ambiguous Structure

**Given inorder: `[1, 2, 3]`**

This could represent any of these different trees:

```
Tree A:          Tree B:          Tree C:
   2                1                3
  / \                \              /
 1   3                2            2
                       \          /
                        3        1
```

All three trees produce the same inorder traversal!

#### Key Insight: Left vs Right Subtrees

**Inorder helps identify subtree boundaries:**

If you know a node's position in inorder traversal, you can determine:
- Everything **to the left** is in the left subtree
- Everything **to the right** is in the right subtree

This is why **preorder + inorder** works so well for reconstruction!

#### Practical Applications

**Inorder is useful for:**
- **Validating BSTs** (check if inorder is sorted)
- **Converting BST to sorted array** (inorder traversal)
- **Tree reconstruction** (when combined with preorder/postorder)
- **Finding predecessors/successors** in BSTs

#### For Binary Search Trees

**Special property:** Inorder traversal of a BST always produces a **sorted sequence**

This makes inorder traversal the go-to method for:
- Extracting sorted data from a BST
- Validating BST property
- Finding kth smallest element

## Walkthrough
**Given:**
- `preorder = [3, 9, 20, 15, 7]`
- `inorder = [9, 3, 15, 20, 7]`

**Goal:** Reconstruct the original binary tree

## Step-by-Step Construction

### Step 1: Identify Root
**Rule:** First element in preorder is always the root

```
preorder = [3, 9, 20, 15, 7]
            ↑
         Root = 3
```

**Create root node:**
```
    3
```

### Step 2: Split Inorder Array
**Rule:** Find root in inorder to separate left and right subtrees

```
inorder = [9, 3, 15, 20, 7]
           ↑  ↑  ↑
        Left  Root  Right

Left subtree inorder: [9]
Right subtree inorder: [15, 20, 7]
```

### Step 3: Split Preorder Array
**Rule:** Use inorder subtree sizes to split preorder

```
Left subtree size = 1 (just node 9)
Right subtree size = 3 (nodes 15, 20, 7)

preorder = [3, 9, 20, 15, 7]
            ↑  ↑  ↑
         Root Left Right

Left subtree preorder: [9]
Right subtree preorder: [20, 15, 7]
```

### Step 4: Recursively Build Left Subtree
**Input:** 
- Left preorder: `[9]`
- Left inorder: `[9]`

**Process:**
1. Root = 9 (first in preorder)
2. Find 9 in inorder: `[] | 9 | []` (no left or right)
3. Create leaf node

```
    3
   /
  9
```

### Step 5: Recursively Build Right Subtree
**Input:**
- Right preorder: `[20, 15, 7]`
- Right inorder: `[15, 20, 7]`

**Process:**
1. Root = 20 (first in preorder)
2. Find 20 in inorder: `[15] | 20 | [7]`

```
    3
   / \
  9   20
```

### Step 6: Build Right Subtree's Left Child
**Input:**
- Left preorder: `[15]`
- Left inorder: `[15]`

**Process:**
1. Root = 15 (single node)
2. No children

```
    3
   / \
  9   20
     /
    15
```

### Step 7: Build Right Subtree's Right Child
**Input:**
- Right preorder: `[7]`
- Right inorder: `[7]`

**Process:**
1. Root = 7 (single node)
2. No children

```
    3
   / \
  9   20
     / \
    15  7
```

## Final Result

**Constructed Tree:**
```
       3
      / \
     9   20
        / \
       15  7
```

**Verification:**
- Preorder traversal: 3 → 9 → 20 → 15 → 7 ✅
- Inorder traversal: 9 → 3 → 15 → 20 → 7 ✅


In [1]:
import { TreeNode } from "../bin/ds/tree.ts";

export function buildTree(preorder: number[], inorder: number[]): TreeNode | null {
  if (!preorder.length || !inorder.length) return null

  const rootValue: number = preorder[0]
  const root = new TreeNode(rootValue)

  // find root in inorder
  const rootIndex: number = inorder.findIndex(val => val === rootValue)

  // split the left and right pre/inorder arrays
  const lInorder: number[] = inorder.slice(0, rootIndex)
  const rInorder: number[] = inorder.slice(rootIndex + 1, inorder.length)
  const lPreorder: number[] = preorder.slice(1, 1 + lInorder.length)
  const rPreorder: number[] = preorder.slice(1 + lInorder.length)

  // recurse on the children
  root.left = buildTree(lPreorder, lInorder)
  root.right = buildTree(rPreorder, rInorder)

  return root;
}


In [2]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("Build Tree - Basic", () => {
  const res = buildTree([3,9,20,15,7], [9,3,15,20,7]);
  assertEquals(res?.val, 3);
});



Build Tree - Basic ... ok (2ms)

ok | 1 passed | 0 failed (3ms)
